In [1]:
import numpy as np
import os
import pandas as pd
import sys

# Config

In [2]:
HYPER_OPT = True 
HYPER_OPT_TIME = 8*60*60
USE_ORIGINAL_DATA = False 
SUBMIT_TO_KAGGLE = False
COMPUTE_NONLOG = False
USE_NONLOG = False

In [3]:
# Define the base directory (where the notebook is running)
base_dir = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Define the data directory
data_dir = os.path.join(base_dir, "data")

model_dir = os.path.join(base_dir, "models")

notebooks_dir = os.path.join(base_dir, "notebooks")

sys.path.append(base_dir)

# Data Extraction

In [4]:
from axyom_utilities.data_extraction import extract_data
from axyom_utilities.training import train_model_cv
from axyom_utilities.wrappers import CatBoostRegressorWrapper
from axyom_utilities.preprocessing import * 
from toolz import pipe
import joblib

def get_oof(X_train, y_train, X_test):
    
    #X_train, y_train, X_test, X_orig, y_orig = extract_data(data_dir, USE_ORIGINAL_DATA, log_transform=False) # IMPORTANT: no log transform

    preprocessor = lambda df: pipe(df,\
        preprocess_dates,
        all_to_string    
    )

    X_train = preprocessor(X_train)
    X_test = preprocessor(X_test)
    
    params = {
        "iterations":500,
        "learning_rate":0.1,
        "depth":6,
        "eval_metric":"RMSE",
        "random_seed":42,
        "verbose":200,
        "task_type":'GPU',
        "l2_leaf_reg" :0.7,
    }
    
    results = train_model_cv(
        model=CatBoostRegressorWrapper(**params),
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        early_stopping_rounds=100,
        cv_splits=5
    )
    
    oof, test = results["oof_preds"], results['test_preds']
    
    joblib.dump((oof, test), "nonlog_feature/nonlog.pkl")
    
    return oof, test



# oof_log = pd.read_csv(os.path.join(notebooks_dir, "Catboost_nonlog\\oof_preds_log.csv"))
# oof_nonlog = pd.read_csv(os.path.join(notebooks_dir, "Catboost_nonlog\\oof_preds_nonlog.csv"))

# test_log = pd.read_csv(os.path.join(notebooks_dir, "Catboost_nonlog\\test_preds_log.csv"))
# test_nonlog = pd.read_csv(os.path.join(notebooks_dir, "Catboost_nonlog\\test_preds_nonlog.csv"))

# X_train["oof_nonlog"] = oof_nonlog + np.random.normal(0, 0.01, oof_log.shape) # noise
# X_test["oof_nonlog"] = test_nonlog + np.random.normal(0, 0.01, oof_log.shape)

# X_test["oof_log"] = test_log
# X_train["oof_log"] = oof_log

In [5]:
X_train, y_train, X_test, X_orig, y_orig = extract_data(data_dir, USE_ORIGINAL_DATA, log_transform=True)
y_train_nonlog = np.expm1(y_train)

if USE_NONLOG:
    if COMPUTE_NONLOG:
        nonlog_oof_preds, nonlog_test_preds = get_oof(X_train, y_train_nonlog, X_test)
    else:
        nonlog_oof_preds, nonlog_test_preds = joblib.load('nonlog_feature/cat_non_loged.pkl')
    
    X_train["nonlog"] = nonlog_oof_preds
    X_test["nonlog"] = nonlog_test_preds

# Data Cleaning

In [6]:
from axyom_utilities.preprocessing import freq_encode, preprocess_dates, clean_categorical, frequency_encode
from toolz import pipe

def feature_engineering(df):
    
    # df = freq_encode(df)
    
    df['MissingHealth'] = df['Health Score'].isna().astype(int)
    df['Claims v Duration'] = df['Previous Claims'] / df['Insurance Duration']
    df['Health vs Claims'] = df['Health Score'] / df['Previous Claims']
    
    def duplicate_int(col, df):
        df[col + ' Integer'] = df[col].apply(lambda x: int(x) if pd.notna(x) else x)
        df[col] = df[col].fillna('None').astype('string')
        
        return df
    
    df = duplicate_int('Health Score', df)
    df = duplicate_int('Credit Score', df)
    df = duplicate_int('Previous Claims', df)
    df = duplicate_int('Annual Income', df)
    
    df['MissingValuesCount'] = df.isna().sum(axis=1)
    
    # df = freq_encode(df, 'Customer Feedback')
    # df = freq_encode(df, 'Marital Status')
    
    return df

preprocessor = lambda df: pipe(df,\
    preprocess_dates,
    clean_categorical,
    feature_engineering,  
)

X_train = preprocessor(X_train)
X_test = preprocessor(X_test)
if USE_ORIGINAL_DATA:
    X_orig = preprocessor(X_orig)
    
frequency_encode(X_train, X_test, drop_org=False) # data leaking but acceptable in this context


(          Age  Gender Annual Income Marital Status  Number of Dependents  \
 id                                                                         
 0        19.0  Female       10049.0        Married                   1.0   
 1        39.0  Female       31678.0       Divorced                   3.0   
 2        23.0    Male       25602.0       Divorced                   3.0   
 3        21.0    Male      141855.0        Married                   2.0   
 4        21.0    Male       39651.0         Single                   1.0   
 ...       ...     ...           ...            ...                   ...   
 1199995  36.0  Female       27316.0        Married                   0.0   
 1199996  54.0    Male       35786.0       Divorced                   NaN   
 1199997  19.0    Male       51884.0       Divorced                   0.0   
 1199998  55.0    Male          None         Single                   1.0   
 1199999  21.0  Female          None       Divorced                   0.0   

# HyperOpt

In [9]:
from axyom_utilities.hyperparameter_tuning import CatBoostTuner

if HYPER_OPT:
    catboost_tuner = CatBoostTuner(X_train, y_train, HYPER_OPT_TIME)
    best_params = catboost_tuner.tune()
    catboost_tuner.plot()
else:
    best_params = {
        "iterations":3000,
        "learning_rate":0.05,
        "depth":6,
        "eval_metric":"RMSE",
        "random_seed":42,
        "verbose":200,
        "task_type":'GPU',
        "l2_leaf_reg" :0.7,
    }

[I 2024-12-22 02:26:40,696] Using an existing study with name 'catboost' instead of creating a new one.


Training fold 1...


[W 2024-12-22 02:27:04,440] Trial 34 failed with parameters: {'learning_rate': 0.005597386228447732, 'depth': 9, 'l2_leaf_reg': 1.1837734915639073, 'bagging_temperature': 0.34044045892358055, 'border_count': 98, 'random_strength': 9.406100214941418, 'min_data_in_leaf': 34} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "c:\Users\Axyom\anaconda3\envs\kaggle_env\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "c:\Users\Axyom\Kaggle\PG S4e12 - Regression Insurance\axyom_utilities\hyperparameter_tuning.py", line 27, in <lambda>
    objective = lambda trial: self.generic_objective(trial, model_generator)
  File "c:\Users\Axyom\Kaggle\PG S4e12 - Regression Insurance\axyom_utilities\hyperparameter_tuning.py", line 42, in generic_objective
    results = train_model_cv(
  File "c:\Users\Axyom\Kaggle\PG S4e12 - Regression Insurance\axyom_utilities\training.py", line 34, in train_model

KeyboardInterrupt: 

In [10]:
best_params

{'iterations': 5883,
 'task_type': 'GPU',
 'verbose': False,
 'learning_rate': 0.00521765446243734,
 'depth': 11,
 'l2_leaf_reg': 0.020973837119135495,
 'bagging_temperature': 0.12952601761069596,
 'border_count': 136,
 'random_strength': 6.979958130250014,
 'min_data_in_leaf': 1}

In [10]:
best_params["iterations"] = 10000

# Model Training

In [ ]:
import json

model = CatBoostRegressorWrapper(**best_params)

# Use the train_model function to train and evaluate the model
results = train_model_cv(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    X_orig=X_orig,
    early_stopping_rounds=100,
    cv_splits=5
)

Training fold 1...
Fold 1 RMSE: 1.0314
Training fold 2...
Fold 2 RMSE: 1.0306
Training fold 3...
Fold 3 RMSE: 1.0296
Training fold 4...
Fold 4 RMSE: 1.0319
Training fold 5...
Fold 5 RMSE: 1.0309
Mean CV RMSE: 1.0309 ± 0.0008


In [12]:
# from sklearn import mean_squared_log_error

# def rmsle(y_true, y_pred):
#     return np.sqrt(mean_squared_log_error(y_true, y_pred))

# rmsle_score = rmsle(y_train, results["oof_preds"])

# print(f"RMSLE score = {rmsle_score}")

In [13]:
models = results['models']

# Get feature importance
feature_importances = models[3].catboost_model_.get_feature_importance(type='PredictionValuesChange')

# Map to feature names
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df)

                    Feature  Importance
33    Annual Income Integer   15.035999
31     Credit Score Integer    8.780135
7              Health Score    8.296790
12             Credit Score    5.498149
30     Health Score Integer    5.378875
40        Health Score_freq    4.669488
36       Annual Income_freq    4.274421
2             Annual Income    4.095164
26           DaysSinceStart    4.084965
10          Previous Claims    4.024697
14        Customer Feedback    2.578604
44        Credit Score_freq    2.277013
0                       Age    1.986682
11              Vehicle Age    1.607041
3            Marital Status    1.405289
20                     Week    1.322552
4      Number of Dependents    1.321311
13       Insurance Duration    1.302202
16       Exercise Frequency    1.251517
19                      Day    1.192000
6                Occupation    1.171781
17            Property Type    1.128039
29         Health vs Claims    1.109188
5           Education Level    1.092786


In [14]:
mean_score = np.mean(results["cv_scores"])
std_score = np.std(results["cv_scores"])

# Saving

In [15]:
import joblib

results_dir = f"{mean_score:.4f}+-{std_score:.4f}"
os.makedirs(results_dir, exist_ok=True)

# Save OOF predictions as a CSV file
oof_preds_df = pd.DataFrame({"oof_preds": results["oof_preds"]})
oof_preds_df.to_csv(os.path.join(results_dir, "oof_preds.csv"), index=False)

test_preds_df = pd.DataFrame({"test_preds": results["test_preds"]})
test_preds_df.to_csv(os.path.join(results_dir, "test_preds.csv"), index=False)

#joblib.dump(results["models"], "models.pkl")

# Submission

In [16]:
y_pred = np.expm1(results['test_preds'])

submission = pd.DataFrame({
    'id': X_test.index,  
    'Premium Amount': y_pred
})

FILE_PATH = os.path.join(results_dir, f"Catboost_{mean_score:.4f}+-{std_score:.4f}.csv")

submission.to_csv(FILE_PATH, index=False)

In [17]:
if True: 
    # Define your message and file paths
    COMP_NAME = "playground-series-s4e12"

    SUBMIT_MESSAGE = "Catboost_{mean_score:.4f}+-{std_score:.4f}"

    # Submit to Kaggle
    os.system(f'kaggle competitions submit -c {COMP_NAME} -f {FILE_PATH} -m "{SUBMIT_MESSAGE}"')